In [1]:
import pandas as pd
from rdkit import Chem

In [2]:
df = pd.read_excel('prism_data.xlsx') #from doi.org/10.1038/s41467-020-19986-1, supplementary data 2

In [3]:
df.drop(columns=['Predicted SMILES', 'Tanimoto coefficient', 'Method'], inplace=True)

In [4]:
df.drop_duplicates(subset=['Cluster'], keep='first', inplace=True)
df.reset_index(drop=True, inplace=True)

In [5]:
df

,Cluster,True SMILES
0,Abyssomicin.FASTA,CC1CC(C(=O)C2=C3C4(CC(C(O3)C(C4C=CC1=O)O)C)OC2...
1,Acidobactin_Cluster.fasta,CC(O)C1NC(=O)CC(O)C(CCCN(O)C=O)NC(=O)CC(C)OC(=...
2,actinonin_BGC0001442.fasta,O=C([C@H](C(C)C)NC([C@@H](CC(NO)=O)CCCCC)=O)N1...
3,Actiphenol.fasta,O=C(CC1CC(NC(C1)=O)=O)C2=C(O)C(C)=CC(C)=C2
4,Aerobactin_partial.fasta,CC(N(O)CCCC[C@H](NC(CC(C(O)=O)(O)CC(N[C@H](C(O...
...,...,...
1276,Totopotensamide_A.fasta,CC[C@H](C)[C@@H]1NC([C@H](NC([C@H](NC([C@H](C)...
1277,vibrioferrin_partial.fasta,CC(N(O)CCCC[C@H](NC(CC(C(O)=O)(O)CC(N[C@H](C(O...
1278,warkmycin.fasta,O[C@@]12[C@@]([C@H](OC(C)=O)C(C)=C[C@@H]2O[C@@...
1279,welwitindolinone.fasta,O=C(NC1=C2C=CC=C1)[C@]2(C(C)(C)[C@]([H])3C[C@H...


In [6]:
def canonicalize_smiles(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    molecule = Chem.MolToSmiles(molecule, canonical=True, isomericSmiles=False)
    return molecule

df['True SMILES'] = df['True SMILES'].apply(canonicalize_smiles)

In [7]:
df['Cluster'] = df['Cluster'].str.replace('.gbk', '.fasta')

In [8]:
df.to_csv('prism_bgc.csv', index=False)

In [9]:
import tarfile
import os
from Bio import SeqIO

In [10]:
os.makedirs('prism_fasta', exist_ok=True)

with tarfile.open('BGCs.tar', 'r') as tar:
    sequences = [m for m in tar.getmembers() if m.name.startswith('./sequences/')]
    
    for member in sequences:
        filename = member.name.split('/')[-1]
        
        file_obj = tar.extractfile(member)
        if file_obj:
            with open(f'prism_fasta/{filename}', 'wb') as f:
                f.write(file_obj.read())

In [11]:
#for each file in prism_fasta, get the gbk, and substitute it with the fasta sequence

gbk_files = [f for f in os.listdir('prism_fasta') if f.endswith('.gbk')]

for gbk in gbk_files:
    gbk_record = SeqIO.read(f'prism_fasta/{gbk}', 'genbank')
    dna_seq = str(gbk_record.seq)
    fasta_filename = gbk.replace('.gbk', '.fasta')
    with open(f'prism_fasta/{fasta_filename}', 'w') as fasta_file:
        fasta_file.write(f'>{gbk_record.id}\n{dna_seq}\n')
    os.remove(f'prism_fasta/{gbk}')

In [12]:
# for each file with more than one sequence, sub for the concatenated sequence
for fasta_file in os.listdir('prism_fasta'):
    records = list(SeqIO.parse(f'prism_fasta/{fasta_file}', 'fasta'))
    if len(records) > 1:
        concatenated_seq = ''.join(str(record.seq) for record in records)
        with open(f'prism_fasta/{fasta_file}', 'w') as f:
            f.write(f'>{records[0].id}\n{concatenated_seq}\n')

In [13]:
# check if file has more than one sequence
for fasta_file in os.listdir('prism_fasta'):
    records = list(SeqIO.parse(f'prism_fasta/{fasta_file}', 'fasta'))
    if len(records) > 1:
        print(f'{fasta_file} has more than one sequence: {len(records)} sequences')

In [14]:
# rename each header in the fasta files to the name of the file
for fasta_file in os.listdir('prism_fasta'):
    new_header = fasta_file
    records = list(SeqIO.parse(f'prism_fasta/{fasta_file}', 'fasta'))
    if records:
        records[0].id = new_header
        records[0].description = ''
        SeqIO.write(records, f'prism_fasta/{fasta_file}', 'fasta')